<a href="https://colab.research.google.com/github/ddundee24/SABR_Level3_Final/blob/main/Beat_Marcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
## Install and read relevant packages

install.packages("tidyverse")
install.packages("Lahman")
library(tidyverse)
library(Lahman)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [13]:
## Read in Data
df <- read.csv("https://huggingface.co/spaces/rkarthur/sabr3evaluation/raw/main/data/SABR3_Final_Assignment_data.csv")
head(df)

,bbref_id,Age,PA1,PA2,PA3,PA4,OPSY1,OPSY2,OPSY3,OPSY4
,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,abreujo02,33,262,693,553,675,0.987,0.8340000,0.798,0.906
2,adriaeh01,30,101,236,366,186,0.557,0.7650000,0.680,0.707
3,aguilje01,30,216,738,566,311,0.809,0.7135691,0.890,0.837
4,ahmedni01,30,217,625,564,178,0.729,0.7530000,0.700,0.717
5,albieoz01,23,124,702,684,244,0.773,0.8520000,0.757,0.810
6,alfarjo01,27,100,465,377,114,0.624,0.7360000,0.731,0.874


In [14]:
## Develop MARCEL projection for OPS
df$ops_next_year <- ((5*df$OPSY1) + (4*df$OPSY2) + (3*df$OPSY3))/12

df$total_PA <- df$PA1 + df$PA2 + df$PA3

df$reliability <- df$total_PA / (df$total_PA + 1200)

df$player_rate <- (df$OPSY1 + df$OPSY2 + df$OPSY3) / 3

df$regressed_rate <- (df$reliability * df$player_rate) + ((1 - df$reliability)*0.720)

df$age_adjustment <- ifelse (df$Age >= 29,(df$Age - 29) * -0.003, (29 - df$Age)*0.006)

df$MARCEL_OPS <- (1 + df$age_adjustment) * df$regressed_rate

head(df)

,bbref_id,Age,PA1,PA2,PA3,PA4,OPSY1,OPSY2,OPSY3,OPSY4,ops_next_year,total_PA,reliability,player_rate,regressed_rate,age_adjustment,MARCEL_OPS
,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,abreujo02,33,262,693,553,675,0.987,0.8340000,0.798,0.906,0.8887500,1508,0.5568685,0.8730000,0.8052009,-0.012,0.7955385
2,adriaeh01,30,101,236,366,186,0.557,0.7650000,0.680,0.707,0.6570833,703,0.3694167,0.6673333,0.7005441,-0.003,0.6984424
3,aguilje01,30,216,738,566,311,0.809,0.7135691,0.890,0.837,0.7974397,1520,0.5588235,0.8041897,0.7670472,-0.003,0.7647460
4,ahmedni01,30,217,625,564,178,0.729,0.7530000,0.700,0.717,0.7297500,1406,0.5395242,0.7273333,0.7239565,-0.003,0.7217846
5,albieoz01,23,124,702,684,244,0.773,0.8520000,0.757,0.810,0.7953333,1510,0.5571956,0.7940000,0.7612325,0.036,0.7886368
6,alfarjo01,27,100,465,377,114,0.624,0.7360000,0.731,0.874,0.6880833,942,0.4397759,0.6970000,0.7098852,0.012,0.7184038


In [15]:
## Calculate error

## Median Absolute Error
marcel_mse <- round(median(abs(df$MARCEL_OPS - df$OPSY4)),5)

## Root Mean Squared Error
marcel_rmse <- round((mean((df$MARCEL_OPS - df$OPSY4)**2)),5)

print(paste("MARCEL MSE:",marcel_mse))
print(paste("MARCEL RMSE:",marcel_rmse))

[1] "MARCEL MSE: 0.06327"
[1] "MARCEL RMSE: 0.0101"


In [16]:
## Beat Marcel Linear Model

## Create Model
  ## Normally this would be where the data is separated into traning and testing data sets, but for this assignment for certification,
  ## We are only concerned with getting the weights (coefficients) for OPS year over year rather than how Marcel weighs them at 5, 4, and 3.

model <- lm(df$OPSY4 ~
          df$OPSY1 + df$OPSY2 + df$OPSY3,
          data = df)
model


Call:
lm(formula = df$OPSY4 ~ df$OPSY1 + df$OPSY2 + df$OPSY3, data = df)

Coefficients:
(Intercept)     df$OPSY1     df$OPSY2     df$OPSY3  
     0.2840       0.1163       0.2314       0.3068  


In [17]:
## With coefficients, we will multiply for projected OPS

df$BETTER_THAN_MARCEL_OPS <- (0.2840 + (0.1163 * df$OPSY1) + (0.2314 * df$OPSY2) + (0.3068 * df$OPSY3))

head(df)

,bbref_id,Age,PA1,PA2,PA3,PA4,OPSY1,OPSY2,OPSY3,OPSY4,ops_next_year,total_PA,reliability,player_rate,regressed_rate,age_adjustment,MARCEL_OPS,BETTER_THAN_MARCEL_OPS
,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,abreujo02,33,262,693,553,675,0.987,0.8340000,0.798,0.906,0.8887500,1508,0.5568685,0.8730000,0.8052009,-0.012,0.7955385,0.8366021
2,adriaeh01,30,101,236,366,186,0.557,0.7650000,0.680,0.707,0.6570833,703,0.3694167,0.6673333,0.7005441,-0.003,0.6984424,0.7344241
3,aguilje01,30,216,738,566,311,0.809,0.7135691,0.890,0.837,0.7974397,1520,0.5588235,0.8041897,0.7670472,-0.003,0.7647460,0.8162586
4,ahmedni01,30,217,625,564,178,0.729,0.7530000,0.700,0.717,0.7297500,1406,0.5395242,0.7273333,0.7239565,-0.003,0.7217846,0.7577869
5,albieoz01,23,124,702,684,244,0.773,0.8520000,0.757,0.810,0.7953333,1510,0.5571956,0.7940000,0.7612325,0.036,0.7886368,0.8033003
6,alfarjo01,27,100,465,377,114,0.624,0.7360000,0.731,0.874,0.6880833,942,0.4397759,0.6970000,0.7098852,0.012,0.7184038,0.7511524


In [18]:
## Calculate Error of Model

## Median Absolute Error
better_than_marcel_mse <- round(median(abs(df$BETTER_THAN_MARCEL_OPS - df$OPSY4)),5)

## Root Mean Squared Error
better_than_marcel_rmse <- round((mean((df$BETTER_THAN_MARCEL_OPS - df$OPSY4)**2)),5)

print(paste("MARCEL MSE:",better_than_marcel_mse))
print(paste("MARCEL RMSE:",better_than_marcel_rmse))

[1] "MARCEL MSE: 0.0581"
[1] "MARCEL RMSE: 0.00795"


In [19]:
write.csv(df,"SABR3_Final")